# .....ILC NK HIPEC.....
Figure 6H-J

This script creates figures present in Figure 6H-J of the manuscript Marchalot et al. "Tumor-infiltrating immature innate lymphoid cells in colorectal cancer are biased towards tissue-resident NK cell/ILC1 differentiation"

Prerequisites: Flow Cytometry processed FCS files in FlowJo and exported as csv files

Input: Flow Cytometry processed FCS files in FlowJo and exported as csv files

Output: plots as in Figure 6H-J in Marchalot et al.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
#Loading csv exported from FLowJo
##exported all fluorophore parameters as channel values (=scaled as shown in scatter plots in FlowJo)
##exported keywords as scaled values and combined both into combined csv file
nILC = pd.read_csv('path/to/merged.csv',
                                delimiter= ";", decimal=".")

nILC

In [ ]:
# Select relevant flow cytometry channels
selected_channels = [
    'IL13',
    'GzmB',
    "GzmK",
    "CD300F",
    "CD49a",
    "IFNg",
    "CD16",
    "NKp80",
    "CD103",
    "IL22",
    "NKp44",
    "CD117",
    "Perforin"
   ]
flow_cytometry_data = nILC[selected_channels]


## Figure 6H UMAP of Colon and CRC nILC post differentiation

In [ ]:
# Pip install umap-learn
import umap

In [ ]:
# Initialize UMAP with desired parameters
umap = umap.UMAP(n_neighbors=15, min_dist=0.2, n_components=2, metric='euclidean')

# Fit and transform the data
umap_result = umap.fit_transform(flow_cytometry_data)

# Create a DataFrame with UMAP results
umap_df = pd.DataFrame(umap_result, columns=['UMAP1', 'UMAP2'])

# Check the UMAP results
#print(umap_df.head())

plt.scatter(umap_df['UMAP1'], umap_df['UMAP2'], s=0.1)
plt.title('UMAP Clustering Results')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')

# Set aspect ratio to be equal
plt.gca().set_aspect('equal', adjustable='box')

plt.show()


In [ ]:
from scipy.stats import gaussian_kde

#plotting the density of datapoints in the UMAP

# Assuming umap_params_df['UMAP_1_FTZE'] and umap_params_df['UMAP_2_FTZE'] are numerical columns
xy = np.vstack([umap_df['UMAP1'], umap_df['UMAP2']])

# Calculate the point density
density = gaussian_kde(xy)(xy)

# Scatter plot with density
plt.scatter(umap_df['UMAP1'], umap_df['UMAP2'], c=density, cmap="turbo", s=10.2, alpha=0.5)

# Set aspect ratio to be equal
plt.gca().set_aspect('equal', adjustable='box')

plt.show()

In [ ]:
# Run Phenograph on the Flow Cytometry data
pip install phenoGraph

In [ ]:
from sklearn.cluster import KMeans
#import matplotlib.pyplot as plt
#import numpy as np


# Calculate the within-cluster sum of squares for different values of k
wcss = []
for k in range(1, 20):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(umap_result) #fit to the umap embedding
    wcss.append(kmeans.inertia_)

# Plot the elbow curve
plt.plot(range(1, 20), wcss, marker='o')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
plt.show()

In [ ]:
#import phenograph libraries necessary for clustering

import pandas as pd
import phenograph
import numpy as np

# Convert the DataFrame to a numpy array

data = flow_cytometry_data.values 
#flow_cytometry data is the selection of flow channels that I previously also used
#for generating the UMAP

# Run Phenograph clustering
labels, graph, Q = phenograph.cluster(np.array(data), k=400) #k determines the number of neighbours used for phenograph clustering

In [ ]:
# Get the number of clusters generated by Phenograph analysis by counting unique values in the labels
num_clusters = len(set(labels))

print("Number of Clusters Identified:", num_clusters)

In [ ]:
# convert the numpy array containing the Phenograph cluster information into a pandas df
phenograph_clusters = pd.DataFrame({"Phenograph_Clusters": labels})
phenograph_clusters
#phenograph_clusters = np.array(data)

In [ ]:
#merge phenograph-cluster data with df containing flow data and UMAP coordinates and Phenograph cluster information:

nILC_UMAP_Phenograph = pd.concat([nILC,umap_df,phenograph_clusters], axis =1)
nILC_UMAP_Phenograph
#save to csv
nILC_UMAP_Phenograph.to_csv("20240814 nILC concat_UMAP_Phenograph.csv", index=False)

In [ ]:
#loading the csv file containing flow data and UMAP and Phenograph data as a pandas df
nILC_UMAP_Phenograph = pd.read_csv("/Users/anne.marchalot/Library/CloudStorage/OneDrive-KarolinskaInstitutet/Karolinska Institute/Bioinfo/FACS Analysis/Lorenz pipeline/20240814 nILC concat_UMAP_Phenograph.csv", 
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap

# Assuming you have a list of unique cluster labels
unique_labels = list(set(labels))


# Define custom colors for each cluster
custom_colors = ['red', 'green', 'blue', 'orange', 'limegreen', 'cornflowerblue', 'deeppink', 'cyan', 'blueviolet', 'gold','salmon','magenta','deeppink']

# Create a custom colormap
custom_colormap = ListedColormap(custom_colors)

# Scatter plot with different colors for each cluster
for i, cluster_label in enumerate(unique_labels):
    cluster_color = custom_colors[i]  # Use the custom colormap directly
    plt.scatter(umap_df.loc[labels == cluster_label, 'UMAP1'],
                umap_df.loc[labels == cluster_label, 'UMAP2'],
                label=f'Cluster {cluster_label}', s=2, c=cluster_color)

# 2D KDE plot across the entire UMAP
sns.kdeplot(x=umap_df['UMAP1'], y=umap_df['UMAP2'],
            color="black", fill=False, alpha=1, thresh=0.2, levels=10, linewidths=0.45)

# Move the legend outside the plot
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', markerscale = 5)

plt.title('UMAP Clustering Results with Cluster Densities')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.gca().set_aspect('equal', adjustable='box')

plt.savefig("Figure_6H.svg", format="svg", dpi=300)
plt.show()


## Figure 6I Heatmap of normalized protein expression by cluster

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.cluster import hierarchy

# Group the data by cluster and calculate mean expression for each marker in each cluster
cluster_means = nILC_UMAP_Phenograph.groupby('Phenograph_Clusters').mean(numeric_only=True)

# Select the markers to plot
markers_to_plot = [
    'IL13',
    'GzmB',
    "GzmK",
    "CD300F",
    "CD49a",
    "IFNg",
    "CD16",
    "NKp80",
    "CD103",
    "IL22",
    "NKp44",
    "CD117",
    "Perforin"
    ]

# Subset the cluster_means DataFrame to include only the selected markers
subset_cluster_means = cluster_means[markers_to_plot]

# Normalize each gene expression individually from -1 to +1
normalized_cluster_means = 2 * (subset_cluster_means - subset_cluster_means.min(axis=0)) / (subset_cluster_means.max(axis=0) - subset_cluster_means.min(axis=0)) - 1

# Create a heatmap using Seaborn with normalized rows and original columns
plt.figure(figsize=(14, 10))

# Heatmap with normalized rows and original columns
sns.heatmap(normalized_cluster_means, cmap='coolwarm', annot=False, fmt=".2f", linewidths=.5, vmin=-1, vmax=1)

# Customize the plot
plt.title('Normalized Gene Expression in Each Cluster')
plt.xlabel('Cluster')
plt.ylabel('Marker')

plt.savefig("nILC-Phenograph_Clusters_MarkerExpression_Normalized_select.svg", format="svg", dpi=300)

plt.show()



## Figure 6J Density difference of Colon vs CRC in naive ILC post differentiation

In [ ]:
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.colors import ListedColormap

# Assuming nILC_UMAP_Phenograph is your DataFrame

# Manually set the range for UMAP1 and UMAP2
umap1_range = np.linspace(-12.4, 12, 100)
umap2_range = np.linspace(-11.5, 10.5, 100)
umap1_grid, umap2_grid = np.meshgrid(umap1_range, umap2_range)

# Calculate density for all time-points on the grid
total_density = gaussian_kde(np.vstack([nILC_UMAP_Phenograph['UMAP1'], 
                                        nILC_UMAP_Phenograph['UMAP2']]))(np.vstack([umap1_grid.ravel(), umap2_grid.ravel()]))
total_density = total_density.reshape(umap1_grid.shape)

# Calculate density for Colon on the grid
density_1 = gaussian_kde(np.vstack([nILC_UMAP_Phenograph[nILC_UMAP_Phenograph['Tissue'] == 'Colon']['UMAP1'], 
                                     nILC_UMAP_Phenograph[nILC_UMAP_Phenograph['Tissue'] == 'Colon']['UMAP2']]))(np.vstack([umap1_grid.ravel(), umap2_grid.ravel()]))
density_1 = density_1.reshape(umap1_grid.shape)

# Calculate density for CRC on the grid
density_2 = gaussian_kde(np.vstack([nILC_UMAP_Phenograph[nILC_UMAP_Phenograph['Tissue'] == 'CRC']['UMAP1'], 
                                     nILC_UMAP_Phenograph[nILC_UMAP_Phenograph['Tissue'] == 'CRC']['UMAP2']]))(np.vstack([umap1_grid.ravel(), umap2_grid.ravel()]))
density_2 = density_2.reshape(umap1_grid.shape)

# Compute the difference in density on the grid
density_difference = density_2 - density_1

# Create a custom colormap for the filled contour plot
cmap = ListedColormap(sns.color_palette("coolwarm", as_cmap=True)(np.linspace(0, 1, 256)))

# Filled contour plot with a color map representing the density differences
contour = plt.contourf(umap1_grid, umap2_grid, density_difference, cmap=cmap, levels=50, extend='both')

# Overlay contour plot based on all time-points
contour_lines = plt.contour(umap1_grid, umap2_grid, total_density, colors='black', linewidths=1)

# Create a mask based on the filled contour plot
mask = contour.collections[0].get_paths()[0].contains_points(np.vstack((umap1_grid.ravel(), umap2_grid.ravel())).T).reshape(umap1_grid.shape)

# Set all values outside the contour lines to NaN
density_difference = np.ma.masked_where(~mask, density_difference)

# Set color bar for the filled contour plot
cbar = plt.colorbar(contour, label='Density Difference')

# Set plot title
plt.title(f'Density Difference: Tissue {'CRC'} - Tissue {'Colon'} with Total Density Overlay')

# Show the plot
plt.show()
